# Disaster Recovery APIs
In this notebook, we will go over the disaster recovery APIs in YBA with examples and the steps required to make those API calls. We assume that you have already two universes deployed in your YBA and you know their universe uuids.
## Get the API Token 
All disaster recovery YBA APIs are restricted to only authenticated users, so to use them through API, you first need to get the API token using the following code:

In [55]:
import requests
import os
from pprint import pprint

yba_url = os.getenv("YBA_URL", "http://192.168.56.102:9000")
yba_user = {
    "email": os.getenv("YBA_USER_EMAIL", "admin"),
    "password": os.getenv("YBA_USER_PASSWORD", "admin"),
}

route = f"{yba_url}/api/v1/api_login"
payload = {
    "email": yba_user["email"],
    "password": yba_user["password"],
}
response = requests.post(url=route, json=payload).json()
pprint(response)

customer_uuid = response["customerUUID"]
yba_api_token = response["apiToken"]
headers = {"X-AUTH-YW-API-TOKEN": yba_api_token}

{'apiToken': '2894987a-4fdf-47cf-95a2-be3f2039cf79',
 'apiTokenVersion': 0,
 'customerUUID': 'f33e3c9b-75ab-4c30-80ad-cba85646ea39',
 'userUUID': 'a9190fe6-1067-409f-95cf-dbd96893c9c9'}


Then you can use `customer_uuid` as a url parameter and pass the `yba_api_token` in the request header with name `X-AUTH-YW-API-TOKEN` to show that the user is authenticated.

## Get Storage Config UUID
Disaster recovery uses backup/restore for replicating existing data on the source universe, and a storage config is required to store the backup and then restore from. To get the storage config uuid, use the following code:

In [56]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/configs"
response = requests.get(url=route, headers=headers).json()
storage_configs = list(filter(lambda config: config["type"] == "STORAGE", response))
if len(storage_configs) < 1:
    print("No storage config found")
    exit(-1)

storage_config_uuid = storage_configs[0]["configUUID"]
print(storage_config_uuid)

301029d7-ebcc-4cd9-9dbb-c87775340d09


## List YSQL DBs on Source Universe
Disaster recovery is only available for YSQL tables and can be set up at the database granularity. You need to pass the list of databases you would like to replicate to the disaster recovery create API.
The following is how you can get the list of the databases for a universe:

In [57]:
source_universe_uuid = os.getenv(
    "YBA_SOURCE_UNIVERSE_UUID", "0194df05-362d-4b73-b9b9-e0e5b3ad02b5"
)
route = f"{yba_url}/api/v1/customers/{customer_uuid}/universes/{source_universe_uuid}/namespaces"
response = requests.get(url=route, headers=headers).json()
ysql_database_list = [
    db
    for db in list(
        filter(lambda db: db["tableType"] == "PGSQL_TABLE_TYPE" and db["name"] != "yugabyte", response)
    )
]

ysql_database_name_list = [db["name"] for db in ysql_database_list]
pprint(ysql_database_name_list)

ysql_database_uuid_list = [db["namespaceUUID"] for db in ysql_database_list]
pprint(ysql_database_uuid_list)

['post', 'postgres']
['0000406e-0000-3000-8000-000000000000', '000033f3-0000-3000-8000-000000000000']


## List Tables on Source Universe
To change the list of tables in replication as part of the disaster recovery config, you the list of the tables UUIDs in the source universe.
Please note that although the API to change the tables list in a disaster recovery is at table granularity, but YBA only supports bootstrapping of YSQL tables with DB granularity, so if you would like to add tables to the disaster recovery config that require bootstrapping, you need to pass all the table UUIDs in a database.
The following is how you can get the list of the tables for a universe:

In [58]:
route = (f"{yba_url}/api/v1/customers/{customer_uuid}/universes/{source_universe_uuid}/tables"
         f"?includeParentTableInfo={str(False).lower()}&onlySupportedForXCluster={str(True).lower()}")
response = requests.get(url=route, headers=headers).json()
ysql_tables = [
    table
    for table in list(
        filter(lambda table: table["tableType"] == "PGSQL_TABLE_TYPE" and table["keySpace"] in ysql_database_name_list, response)
    )
]
pprint(ysql_tables)

ysql_table_uuid_list = [table["tableUUID"] for table in ysql_tables]

[{'colocated': False,
  'isIndexTable': True,
  'keySpace': 'post',
  'pgSchemaName': 'public',
  'relationType': 'INDEX_TABLE_RELATION',
  'sizeBytes': 0.0,
  'tableID': '0000406e000030008000000000004079',
  'tableName': 'postgresqlkeyvalue_v_idx',
  'tableType': 'PGSQL_TABLE_TYPE',
  'tableUUID': '0000406e-0000-3000-8000-000000004079',
  'walSizeBytes': 0.0},
 {'colocated': False,
  'isIndexTable': False,
  'keySpace': 'post',
  'pgSchemaName': 'public',
  'relationType': 'USER_TABLE_RELATION',
  'sizeBytes': 0.0,
  'tableID': '0000406e000030008000000000004074',
  'tableName': 'postgresqlkeyvalue1',
  'tableType': 'PGSQL_TABLE_TYPE',
  'tableUUID': '0000406e-0000-3000-8000-000000004074',
  'walSizeBytes': 0.0},
 {'colocated': False,
  'isIndexTable': False,
  'keySpace': 'post',
  'pgSchemaName': 'public',
  'relationType': 'USER_TABLE_RELATION',
  'sizeBytes': 0.0,
  'tableID': '0000406e00003000800000000000406f',
  'tableName': 'postgresqlkeyvalue',
  'tableType': 'PGSQL_TABLE_TYPE'

## Waiting For Tasks
The disaster recovery APIs will create a task in the backend and returns a task uuid which you can follow to see the progress and the status of the task. You can use the following function to wait for a task:

In [59]:
import time

def waitForTask(task_uuid):
    route = f"{yba_url}/api/v1/customers/{customer_uuid}/tasks/{task_uuid}"
    while True:
        response = requests.get(url=route, headers=headers).json()
        status = response["status"]
        if status == "Failure":
            route = f"{yba_url}/api/customers/{customer_uuid}/tasks/{task_uuid}/failed"
            response = requests.get(url=route, headers=headers)
            if response is not None:
                response = response.json()
                if "failedSubTasks" in response:
                    errors = [
                        subtask["errorString"] for subtask in response["failedSubTasks"]
                    ]
                    print(f"Task {task_uuid} failed with the following errors:")
                    print("\n".join(errors))
                else:
                    pprint(response)
            else:
                print(
                    f"Task {task_uuid} failed, but could not get the failure messages"
                )
            exit(-1)
        elif status == "Success":
            print(f"Task {task_uuid} finished successfully")
            break
        print(f"Waiting for task {task_uuid}...")
        time.sleep(20)

## Creating Disaster Recovery Configs
Now we have all the required information to successfully create a disaster recovery config from universe `source_universe_uuid` to `target_universe_uuid`.

In [60]:
target_universe_uuid = os.getenv(
    "YBA_TARGET_UNIVERSE_UUID", "dc7940d0-8130-4f04-a004-918dd4f4ff95"
)
route = f"{yba_url}/api/v1/customers/{customer_uuid}/dr_configs"
payload = {
    "sourceUniverseUUID": source_universe_uuid,
    "targetUniverseUUID": target_universe_uuid,
    "name": "my-dr",
    "dbs": ysql_database_uuid_list,
    "bootstrapParams": {
        "backupRequestParams": {"storageConfigUUID": storage_config_uuid},
    }
}
response = requests.post(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

dr_config_uuid = response["resourceUUID"]

{'resourceUUID': '98d823ab-d035-475f-868d-e19db1343754',
 'taskUUID': 'ef1f52fd-a0aa-4684-a35b-0f61f0d97d45'}
Waiting for task ef1f52fd-a0aa-4684-a35b-0f61f0d97d45...
Task ef1f52fd-a0aa-4684-a35b-0f61f0d97d45 finished successfully


## Getting Disaster Recovery Configs
You can get the disaster recovery config using its uuid. See the following example.

In [61]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/dr_configs/{dr_config_uuid}"
response = requests.get(url=route, headers=headers).json()
pprint(response)

{'createTime': '2023-12-04T17:39:38Z',
 'drReplicaUniverseActive': False,
 'drReplicaUniverseState': 'Receiving data, Ready for reads',
 'drReplicaUniverseUuid': 'dc7940d0-8130-4f04-a004-918dd4f4ff95',
 'modifyTime': '2023-12-04T17:39:38Z',
 'name': 'my-dr',
 'paused': False,
 'pitrConfigs': [{'createTime': '2023-12-04T17:37:31Z',
                  'customerUUID': 'f33e3c9b-75ab-4c30-80ad-cba85646ea39',
                  'dbName': 'post',
                  'maxRecoverTimeInMillis': 0,
                  'minRecoverTimeInMillis': 0,
                  'retentionPeriod': 259200,
                  'scheduleInterval': 3600,
                  'tableType': 'PGSQL_TABLE_TYPE',
                  'updateTime': '2023-12-04T17:37:31Z',
                  'usedForXCluster': True,
                  'uuid': '10faf9a9-8e67-484c-b692-d65b1956a6ae'},
                 {'createTime': '2023-12-04T17:37:47Z',
                  'customerUUID': 'f33e3c9b-75ab-4c30-80ad-cba85646ea39',
                  'dbName':

## Modifying Tables in Disaster Recovery Configs
You can add/remove tables to/from an existing disaster recovery config. This is useful when you would like to add new tables to your database after the disaster recovery config is set up, or you would like to drop a table from your database.
Please note that to drop a table from your database, *first you need to remove that table from the disaster recovery config.*
To modify the tables in replication in a disaster recovery config, you need to pass the list of the tables that you would like to be in replication. In other words, you remove the table UUIDs that you do not want to be replicated, and add the new table uuids you want to replication. See the following example. 

In [62]:
# Remove tables.
route = f"{yba_url}/api/v1/customers/{customer_uuid}/dr_configs/{dr_config_uuid}/set_tables"
payload = {
    "tables": ysql_table_uuid_list[:-1]
}
response = requests.post(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

# Add tables.
route = f"{yba_url}/api/v1/customers/{customer_uuid}/dr_configs/{dr_config_uuid}/set_tables"
payload = {
    "tables": ysql_table_uuid_list,
    "bootstrapParams": {
        "backupRequestParams": {"storageConfigUUID": storage_config_uuid},
    },
}
response = requests.post(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

{'resourceUUID': '98d823ab-d035-475f-868d-e19db1343754',
 'taskUUID': '3805cff8-5699-4795-80a0-4958dd1d22b6'}
Waiting for task 3805cff8-5699-4795-80a0-4958dd1d22b6...
Task 3805cff8-5699-4795-80a0-4958dd1d22b6 finished successfully
{'resourceUUID': '98d823ab-d035-475f-868d-e19db1343754',
 'taskUUID': 'dc14ca23-b800-4a11-8a8a-5e4343c707f6'}
Waiting for task dc14ca23-b800-4a11-8a8a-5e4343c707f6...
Task dc14ca23-b800-4a11-8a8a-5e4343c707f6 finished successfully


## Reconciling Disaster Recovery Configs with YBDB State
Sometimes, it is required to make changes to the replication group using yb-admin. In these cases, the corresponding disaster recovery config in YBA will not be automatically updated to learn about the yb-admin changes, and a manual synchronization call is required as follows.
Please note that a disaster recovery config named `<dr-name>`, the corresponding replication group name will be `<source-universe-uuid>_--DR-CONFIG-<dr-name>-0`.

In [63]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/dr_configs/{dr_config_uuid}/sync"
response = requests.post(url=route, headers=headers).json()
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

Waiting for task 0f1f0171-4a88-4c73-8c47-698164c09bdd...
Task 0f1f0171-4a88-4c73-8c47-698164c09bdd finished successfully


## Restarting Disaster Recovery Configs
The replication between two universes can break for various reasons including temporary network partitions. In these cases, after the issue is resolved, you can restart replication. You may also include index tables to the replication by restarting the replication for their main tables.

In [64]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/dr_configs/{dr_config_uuid}/restart"
payload = {
    "dbs": ysql_database_uuid_list,
    "bootstrapParams": {
        "backupRequestParams": {"storageConfigUUID": storage_config_uuid},
    },
}
response = requests.post(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

{'resourceUUID': '98d823ab-d035-475f-868d-e19db1343754',
 'taskUUID': 'eff97317-ef8b-485d-8d1c-3c9ca59f30bb'}
Waiting for task eff97317-ef8b-485d-8d1c-3c9ca59f30bb...
Task eff97317-ef8b-485d-8d1c-3c9ca59f30bb finished successfully


## Doing Switchover on Disaster Recovery Configs
You may switch over the primary and dr replica universes and then route your application writes to the old dr replica with zero RPO to drill a failover operation. The `primaryUniverseUuid` field in the payload will be the new primary universe UUID and the `drReplicaUniverseUuid` field will be the new dr replication universe UUID.

In [65]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/dr_configs/{dr_config_uuid}/switchover"
payload = {
    "primaryUniverseUuid": target_universe_uuid,
    "drReplicaUniverseUuid": source_universe_uuid,
}
response = requests.post(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

{'resourceUUID': '98d823ab-d035-475f-868d-e19db1343754',
 'taskUUID': '1d9da05f-7a0b-46b8-a6f3-0852ac0ebd5c'}
Waiting for task 1d9da05f-7a0b-46b8-a6f3-0852ac0ebd5c...
Waiting for task 1d9da05f-7a0b-46b8-a6f3-0852ac0ebd5c...
Task 1d9da05f-7a0b-46b8-a6f3-0852ac0ebd5c finished successfully


## Doing Failover on Disaster Recovery Configs
In case the current primary universe becomes unavailable, you do a failover operations in order to promote the current dr replica as primary and then route your application traffic to the new primary universe. In a failover operation, some data can be lost. To get an estimate of the amount of data that could be lost, you need to get the current safetime using the following api call.

In [66]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/dr_configs/{dr_config_uuid}/safetime"
response = requests.get(url=route, headers=headers).json()
pprint(response)

namespace_id_to_safetime_epoch_micros_dict = {safetime["namespaceId"]: safetime["safetimeEpochUs"] for safetime in response["safetimes"]}

{'safetimes': [{'estimatedDataLossMs': 321,
                'namespaceId': '0000406e000030008000000000000000',
                'namespaceName': 'post',
                'safetimeEpochUs': 1701711850090902,
                'safetimeLagUs': 883087,
                'safetimeSkewUs': 41},
               {'estimatedDataLossMs': 321,
                'namespaceId': '000033f3000030008000000000000000',
                'namespaceName': 'postgres',
                'safetimeEpochUs': 1701711850090821,
                'safetimeLagUs': 883168,
                'safetimeSkewUs': 142}]}


If the estimatedDataLossMs and current safetime on the current dr replica sounds good, you may call the following api do a failover operation (please note that in previous section we switched the source and target universes).

In [67]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/dr_configs/{dr_config_uuid}/failover"
payload = {
    "primaryUniverseUuid": source_universe_uuid,
    "drReplicaUniverseUuid": target_universe_uuid,
    "namespaceIdSafetimeEpochUsMap": namespace_id_to_safetime_epoch_micros_dict
}
response = requests.post(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

{'resourceUUID': '98d823ab-d035-475f-868d-e19db1343754',
 'taskUUID': 'a5f769b5-c1a1-4794-8d7a-5b12225980c6'}
Waiting for task a5f769b5-c1a1-4794-8d7a-5b12225980c6...
Waiting for task a5f769b5-c1a1-4794-8d7a-5b12225980c6...
Task a5f769b5-c1a1-4794-8d7a-5b12225980c6 finished successfully


## Replacing dr replica on Disaster Recovery Configs
After a failover operation, you could either restart the disaster recovery config to use the old primary universe as the dr replica, or you could use the following API to use a new universe as the dr replica.

In [68]:
new_target_universe_uuid = os.getenv(
    "YBA_NEW_TARGET_UNIVERSE_UUID", "fcd8ad18-9130-45d7-b504-fa28d187df05"
)

route = f"{yba_url}/api/v1/customers/{customer_uuid}/dr_configs/{dr_config_uuid}/replace_replica"
payload = {
    "primaryUniverseUuid": source_universe_uuid,
    "drReplicaUniverseUuid": new_target_universe_uuid,
    "bootstrapParams": {
        "backupRequestParams": {"storageConfigUUID": storage_config_uuid},
    },
}
response = requests.post(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

{'resourceUUID': '98d823ab-d035-475f-868d-e19db1343754',
 'taskUUID': 'da3c30e4-e734-4d4f-a0c3-6d5943dc07eb'}
Waiting for task da3c30e4-e734-4d4f-a0c3-6d5943dc07eb...
Task da3c30e4-e734-4d4f-a0c3-6d5943dc07eb finished successfully


## Deleting Disaster Recovery Configs
You can delete the disaster recovery config so there is no replication relation between the two universes and the dr replica will be in active state. Please note that `isForceDelete` is useful when one of the universes is not available or there is an issue with the config. In those cases, you pass `True` and it will ignore errors and delete the config.

In [69]:
route = (f"{yba_url}/api/v1/customers/{customer_uuid}/dr_configs/{dr_config_uuid}"
         f"?isForceDelete={str(False).lower()}")
response = requests.delete(url=route, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

{'resourceUUID': '98d823ab-d035-475f-868d-e19db1343754',
 'taskUUID': 'dc0f6aa2-ceba-47df-a931-0fca8418848b'}
Waiting for task dc0f6aa2-ceba-47df-a931-0fca8418848b...
Task dc0f6aa2-ceba-47df-a931-0fca8418848b finished successfully
